<a href="https://colab.research.google.com/github/esdrasantos/statistical-concepts-and-practice/blob/main/cstfm_mod_9_anova.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [238]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [239]:
import numpy as np
import pandas as pd
import os
import re
import datetime

# Buscando dados de tabelas .csv hospedadas no google drive⚡

In [240]:
dir = '/content/drive/MyDrive/faturas_nu_csv'

arquivos = [arquivo for arquivo in os.listdir(dir)]

padrao = f"_(.*?)-"

arquivosord = sorted(arquivos, key=lambda x: int(re.findall(padrao, x)[0]))
arquivosord = sorted(arquivosord, key=lambda x: int(x.split('-')[1]))
arquivosord

['Nubank_2025-01-11.csv',
 'Nubank_2025-02-11.csv',
 'Nubank_2025-03-11.csv',
 'Nubank_2025-04-11.csv',
 'Nubank_2025-05-11.csv',
 'Nubank_2025-06-11.csv',
 'Nubank_2025-07-11.csv',
 'Nubank_2025-08-11.csv',
 'Nubank_2025-09-11.csv',
 'Nubank_2025-10-11.csv',
 'Nubank_2025-11-11.csv']

In [241]:
dados = pd.DataFrame()

for arquivo in arquivosord:
  tabela = pd.read_csv(f'{dir}/{arquivo}')
  tabela['Arquivo Origem'] = arquivo
  dados = pd.concat([dados,tabela])

dados.reset_index(drop=True,inplace=True)

In [242]:
dados['Fech. Fatura'] = dados['Arquivo Origem']\
.str.rsplit('_', n=1).str[1]\
.str.split('.').str[0]

# Corrigindo as datas das observações e preparando dados ⛴

In [243]:
dia_mes = dados['Data'].str.split(' ')
dados['Dia'] = dia_mes.str[0].str.strip()
dados['Mes'] = dia_mes.str[1].str.strip()
dados['Ano'] = dados['Fech. Fatura'].str\
              .split('-').str[0]\
              .str.strip()
meses = {
    "JAN": 1,
    "FEV": 2,
    "MAR": 3,
    "ABR": 4,
    "MAI": 5,
    "JUN": 6,
    "JUL": 7,
    "AGO": 8,
    "SET": 9,
    "OUT": 10,
    "NOV": 11,
    "DEZ": 12
}

dados['N Mes'] = dados['Mes'].map(meses)

dados['Data_Compra_Aux'] = dados.Ano + "-"\
                           + dados['N Mes'].astype(str) + "-"\
                           + dados.Dia

In [244]:
dados.drop(columns=['Dia','Mes','Ano','N Mes'],inplace=True)

In [245]:
dtdiff = pd.to_datetime(dados['Fech. Fatura'])-pd.to_datetime(dados['Data_Compra_Aux'])
dados['Dias_Compra'] = dtdiff.dt.days

In [246]:
dados.drop(columns=["Data"], inplace=True)

In [247]:
from dateutil.relativedelta import relativedelta

In [248]:
dados['Data'] = pd.to_datetime(dados['Data_Compra_Aux'])
filtro = dados['Dias_Compra'] < 0
dados.loc[filtro, 'Data'] = dados.loc[filtro, 'Data'] - pd.DateOffset(years=1)

In [249]:
dados.drop(columns=['Data_Compra_Aux', 'Dias_Compra'],inplace=True)

In [250]:
dados['Mes Compra'] = dados['Data'].dt.month
dados['Semana Compra'] = dados['Data'].dt.isocalendar().week

In [251]:
def semana_do_mes(data):
    return data.day // 7 + 1

semana_do_mes(pd.to_datetime('2025-12-31'))

5

In [252]:
datas = dados[['Data']].copy()
print(datas.head())


        Data
0 2024-12-01
1 2024-12-04
2 2024-12-04
3 2024-12-04
4 2024-12-04


In [253]:
datas['Semana do Mes'] = datas.apply(lambda x: semana_do_mes(pd.to_datetime(x['Data'])), axis=1)

print(datas[-15:])

           Data  Semana do Mes
1055 2025-10-29              5
1056 2025-10-30              5
1057 2025-10-31              5
1058 2025-10-31              5
1059 2025-10-31              5
1060 2025-10-31              5
1061 2025-10-31              5
1062 2025-10-31              5
1063 2025-10-31              5
1064 2025-10-31              5
1065 2025-11-01              1
1066 2025-11-01              1
1067 2025-11-02              1
1068 2025-11-02              1
1069 2025-11-03              1


In [254]:
dados['Semana do Mes'] = datas['Semana do Mes']
dados['Ano'] = datas['Data'].dt.year
dados['Mes'] = datas['Data'].dt.month
print(dados.head())

                             Compra     Valor             Categoria  \
0                        Uber* Trip  R$ 12,99            Transporte   
1  Amazon Marketplace - Parcela 2/7  R$ 53,28  Shopping e ecommerce   
2    Saoleopoldomeias - Parcela 2/4  R$ 75,50  Shopping e ecommerce   
3  Amazon Marketplace - Parcela 7/8  R$ 53,13  Shopping e ecommerce   
4  Amazon Marketplace - Parcela 3/6  R$ 55,20  Shopping e ecommerce   

          Arquivo Origem Fech. Fatura       Data  Mes Compra  Semana Compra  \
0  Nubank_2025-01-11.csv   2025-01-11 2024-12-01          12             48   
1  Nubank_2025-01-11.csv   2025-01-11 2024-12-04          12             49   
2  Nubank_2025-01-11.csv   2025-01-11 2024-12-04          12             49   
3  Nubank_2025-01-11.csv   2025-01-11 2024-12-04          12             49   
4  Nubank_2025-01-11.csv   2025-01-11 2024-12-04          12             49   

   Semana do Mes   Ano  Mes  
0              1  2024   12  
1              1  2024   12  
2       

In [255]:
dados.drop(columns=['Arquivo Origem', 'Fech. Fatura', 'Compra'],inplace=True)
print(dados.head())

      Valor             Categoria       Data  Mes Compra  Semana Compra  \
0  R$ 12,99            Transporte 2024-12-01          12             48   
1  R$ 53,28  Shopping e ecommerce 2024-12-04          12             49   
2  R$ 75,50  Shopping e ecommerce 2024-12-04          12             49   
3  R$ 53,13  Shopping e ecommerce 2024-12-04          12             49   
4  R$ 55,20  Shopping e ecommerce 2024-12-04          12             49   

   Semana do Mes   Ano  Mes  
0              1  2024   12  
1              1  2024   12  
2              1  2024   12  
3              1  2024   12  
4              1  2024   12  


# Definindo dimensões para ANOVA 1 VIA

In [256]:
df = dados.copy()
df['Valor'] = df['Valor'].astype(str)
df['Valor'] = df['Valor'].str.replace('R$', '')
df['Valor'] = df['Valor'].str.replace(',', '.')
df['Valor'] = df['Valor'].str.replace(r'\.(?=.*\.)', '', regex=True)

In [257]:
df['Valor'] = pd.to_numeric(df['Valor'], errors='coerce')


In [258]:
df['Valor'].agg(['min','max','mean'])

,Valor
min,0.000000
max,1011.550000
mean,50.049262


In [259]:
print(df.tail())

       Valor   Categoria       Data  Mes Compra  Semana Compra  Semana do Mes  \
1065   31.00  Transporte 2025-11-01          11             44              1   
1066   17.90  Lavanderia 2025-11-01          11             44              1   
1067    5.87  Transporte 2025-11-02          11             44              1   
1068  126.86    Mercados 2025-11-02          11             44              1   
1069  131.72       Ifood 2025-11-03          11             45              1   

       Ano  Mes  
1065  2025   11  
1066  2025   11  
1067  2025   11  
1068  2025   11  
1069  2025   11  


In [260]:
quintas = df[df['Data'].dt.weekday == 3]

In [261]:
# media_quintas = quintas.groupby(['Ano', 'Mes', 'Semana do Mes'])['Valor'].mean().reset_index()

quintas.drop(columns=['Categoria', 'Data', 'Semana Compra', 'Mes Compra'], inplace=True)

/tmp/ipython-input-2313024712.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quintas.drop(columns=['Categoria', 'Data', 'Semana Compra', 'Mes Compra'], inplace=True)


In [262]:
print(quintas.tail())

       Valor  Semana do Mes   Ano  Mes
1007   37.71              2  2025   10
1008   69.98              2  2025   10
1009   29.00              2  2025   10
1041  120.38              4  2025   10
1056   33.80              5  2025   10


In [263]:
media_quintas = quintas.copy()

media_quintas['Ano Mes'] = quintas['Ano'].astype(str) + '-' + quintas['Mes'].astype(str)

media_quintas.drop(columns=['Ano', 'Mes'], inplace=True)

# ANOVA DE 1 VIA

Repondamos a seguinte pergunta: \
As médias de gasto nas quintas-feiras de cada mês são estatisticamente iguais? \
Se forem diferentes, em quais meses essas médias são estatisticamente diferentes? \

Ho: μ1=μ2=μ3=...=μ12 \
Ha: ∃ μi, μj | μi != μj

ou, simplesmente \

Ho: μ1=μ2=μ3=...=μ12 \
Ha: Existe ao menos uma média diferente das demais.


In [274]:
print(media_quintas.tail())  # Ultimos 5

      Valor  Semana do Mes  Ano Mes
152   37.71              2  2025-10
153   69.98              2  2025-10
154   29.00              2  2025-10
155  120.38              4  2025-10
156   33.80              5  2025-10


In [292]:
g_sum_quintas = media_quintas.groupby(['Ano Mes', 'Semana do Mes'])['Valor'].sum()
print(g_sum_quintas.head())

Ano Mes  Semana do Mes
2024-12  2                139.80
         3                238.49
         4                 48.01
2025-1   2                 64.97
         3                198.30
Name: Valor, dtype: float64


In [281]:
from scipy.stats import f_oneway

In [284]:
# Tratamento de NAN

g_sum_quintas.dropna(inplace=True)

In [294]:
c2412 = g_sum_quintas['2024-12']
c2501 = g_sum_quintas['2025-1']
c2502 = g_sum_quintas['2025-2']
c2503 = g_sum_quintas['2025-3']
c2504 = g_sum_quintas['2025-4']
c2505 = g_sum_quintas['2025-5']
c2506 = g_sum_quintas['2025-6']
c2507 = g_sum_quintas['2025-7']
c2508 = g_sum_quintas['2025-8']
c2509 = g_sum_quintas['2025-9']
c2510 = g_sum_quintas['2025-10']

print(c2510, c2509)

Semana do Mes
1    112.45
2    205.16
4    120.38
5     33.80
Name: Valor, dtype: float64 Semana do Mes
1    487.33
2    130.88
3    440.84
4    114.04
Name: Valor, dtype: float64


In [290]:
f_stat, p_value = f_oneway(c2412, c2501, c2502, c2503, c2504, c2505, c2506, c2507, c2508, c2509, c2510)

print("F-statistic:", f_stat)
print("p-value:", p_value)

F-statistic: 0.7690693789639272
p-value: 0.6566107649924653


In [293]:
# Decisão:

if p_value < 0.05:
    print("As médias de gasto nas quintas-feiras de cada mês são estatisticamente diferentes.")
else:
    print("As médias de gasto nas quintas-feiras de cada mês são estatisticamente iguais.")

As médias de gasto nas quintas-feiras de cada mês são estatisticamente iguais.


# Pivot dos Dados


In [285]:
c202412 = media_quintas[media_quintas['Ano Mes'] == '2024-12']['Valor']
c202501 = media_quintas[media_quintas['Ano Mes'] == '2025-1']['Valor']
c202502 = media_quintas[media_quintas['Ano Mes'] == '2025-2']['Valor']
c202503 = media_quintas[media_quintas['Ano Mes'] == '2025-3']['Valor']
c202504 = media_quintas[media_quintas['Ano Mes'] == '2025-4']['Valor']
c202505 = media_quintas[media_quintas['Ano Mes'] == '2025-5']['Valor']
c202506 = media_quintas[media_quintas['Ano Mes'] == '2025-6']['Valor']
c202507 = media_quintas[media_quintas['Ano Mes'] == '2025-7']['Valor']
c202508 = media_quintas[media_quintas['Ano Mes'] == '2025-8']['Valor']
c202509 = media_quintas[media_quintas['Ano Mes'] == '2025-9']['Valor']
c202510 = media_quintas[media_quintas['Ano Mes'] == '2025-10']['Valor']

print(c202412.head())

0    58.90
1    57.48
2    23.42
3    19.90
4    84.34
Name: Valor, dtype: float64


In [268]:
f_stat, p_value = f_oneway(c202412, c202501, c202502, c202503, c202504, c202505, c202506, c202507, c202508, c202509, c202510)

print("F-statistic:", f_stat)
print("p-value:", p_value)

F-statistic: 1.3420574800983338
p-value: 0.2134597328591055


In [269]:
media_quintas.reset_index(drop=True, inplace=True)
print(media_quintas.tail())

      Valor  Semana do Mes  Ano Mes
152   37.71              2  2025-10
153   69.98              2  2025-10
154   29.00              2  2025-10
155  120.38              4  2025-10
156   33.80              5  2025-10


In [270]:
# Só para ficar igual ao exercício do curso - com o fator nas colunas e o valor nas linhas.

cumcount = media_quintas.groupby('Ano Mes').cumcount()

pv_quintas = media_quintas.set_index(cumcount).pivot(columns='Ano Mes', values='Valor')

print("Head:\n", pv_quintas.head())

print("Tail:\n", pv_quintas.tail())

Head:
 Ano Mes  2024-12  2025-1  2025-10  2025-2  2025-3  2025-4  2025-5  2025-6  \
0          58.90    9.49   112.45  704.41   30.75    9.65   16.90   17.90   
1          57.48   55.48    11.95   10.79  326.08    8.94   16.90   48.36   
2          23.42   12.95    48.56   11.56    9.69   35.00  198.38   17.90   
3          19.90   30.00     7.96  109.89   10.98   24.90   68.99   16.90   
4          84.34    7.00    37.71   27.68   94.90   14.97    5.00    5.12   

Ano Mes  2025-7  2025-8  2025-9  
0         24.90   23.26   82.52  
1         44.90   34.04   67.10  
2         66.76   16.90   32.02  
3        327.45  124.61   55.44  
4          2.47   17.90   60.90  
Tail:
 Ano Mes  2024-12  2025-1  2025-10  2025-2  2025-3  2025-4  2025-5  2025-6  \
20           NaN     NaN      NaN     NaN     NaN     NaN   70.81    74.0   
21           NaN     NaN      NaN     NaN     NaN     NaN   84.54     NaN   
22           NaN     NaN      NaN     NaN     NaN     NaN   75.88     NaN   
23         

In [271]:
# Tudo isso para porque f_oneaway anova não suporta NaN
# E também tive de pivotar os dados para ficar similar à
# prática do curso...
# O arquivo <> tem um exemplo mais simples

dfs = {
    col: pv_quintas[col].dropna().to_list()
    for col in pv_quintas.columns
}

In [272]:
[df for df in dfs.values()][:2]

[[58.9, 57.48, 23.42, 19.9, 84.34, 29.99, 104.26, 48.01],
 [9.49,
  55.48,
  12.95,
  30.0,
  7.0,
  104.78,
  35.07,
  8.5,
  103.58,
  317.81,
  121.0,
  102.39]]

In [273]:
f_stat, p_value = f_oneway(
    *[
        df
        for df in dfs.values()
    ]
)
print("F-statistic:", f_stat)
print("p-value:", p_value)

F-statistic: 1.3420574800983336
p-value: 0.2134597328591055
